In [1]:
#Fixed problem with CUDA and TF2
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import pandas as pd
import numpy as np
import os
import tensorflow.keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.applications.vgg16 import preprocess_input, VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [3]:
base_model = VGG16(weights='imagenet',include_top=False) 

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation='relu')(x) 
x = Dense(1024,activation='relu')(x) 
x = Dense(512,activation='relu')(x) 
preds = Dense(4,activation='softmax')(x)

In [4]:
model = Model(inputs=base_model.input, outputs=preds)

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [6]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 global_average_pooling2d
20 dense
21 dense_1
22 dense_2
23 dense_3


In [7]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [24]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) 
train_generator=train_datagen.flow_from_directory('./dataset/training_set/', 
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=2,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 1951 images belonging to 4 classes.


In [28]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) 
test_set = train_datagen.flow_from_directory('./dataset/test_set/', 
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=2,
                                                 class_mode='categorical')

Found 656 images belonging to 4 classes.


In [29]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy



In [30]:
step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=5)

Epoch 1/5
487/487 [==============================] - 340s 698ms/step - loss: 0.1103 - accuracy: 0.9594
Epoch 2/5
487/487 [==============================] - 336s 690ms/step - loss: 1.1504e-06 - accuracy: 1.0000
Epoch 3/5
487/487 [==============================] - 336s 690ms/step - loss: 4.3711e-07 - accuracy: 1.0000
Epoch 4/5
487/487 [==============================] - 337s 691ms/step - loss: 2.5347e-07 - accuracy: 1.0000
Epoch 5/5
487/487 [==============================] - 337s 691ms/step - loss: 1.9968e-07 - accuracy: 1.0000


In [31]:
step_size_test=test_set.n//test_set.batch_size
score = model.evaluate_generator(test_set, steps=step_size_test)

In [32]:
print(model.metrics_names)
print(score)

['loss', 'accuracy']
[0.002563542110016596, 0.9984756]


In [33]:
step_size_test=test_set.n//test_set.batch_size
pred = model.predict_generator(test_set, steps=step_size_test)

In [34]:
pred

array([[1.9921300e-09, 2.4628496e-12, 1.0000000e+00, 4.9207435e-12],
       [3.1497088e-08, 1.0000000e+00, 1.0058935e-12, 1.0480702e-09],
       [4.9756463e-11, 1.0000000e+00, 1.1808810e-13, 1.1267455e-11],
       ...,
       [1.0000000e+00, 3.9036253e-12, 1.7941774e-12, 2.9874809e-12],
       [1.4990736e-09, 1.5754753e-12, 1.0000000e+00, 3.5965998e-12],
       [1.0000000e+00, 6.9814853e-12, 1.9270766e-11, 7.0182818e-12]],
      dtype=float32)

In [22]:
test_set[0][1]

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]], dtype=float32)

In [62]:
import pandas as pd

In [65]:
df = pd.DataFrame(pred)

In [67]:
df.apply(round)

,0,1,2,3
0,1.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0
6,1.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0
8,0.0,0.0,1.0,0.0
9,0.0,0.0,0.0,1.0
